In [86]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import cv2
import os

In [87]:
# Carrega imagens de entrada
image_names = os.listdir('input')
images = []

for image_name in image_names:
    images.append({"name": image_name, "image": mpimg.imread('input/' + image_name)})

In [88]:
import math
import statistics


def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    ignore_mask_color = 255
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    # Retorna onde os pixels não são zero 
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines):
    # Define a cor das linhas
    color=[255, 0, 255]
    # Define a grossura da linha sendo desenhada
    thickness=3
    """
    Essa função é responsável por encontrar as linhas na imagem. 
    Separar segmentos de linha por sua inclinação partindo de duas cordenadas ((y2-y1)/(x2-x1)) para decidir quais segmentos 
    fazem parte da linha à esquerda e à direita. 
    Calcular a média da posição de cada uma das linhas e extrapolar para o topo e a base da pista.
    """
    left_lines = []
    right_lines = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            # Desenha uma linha a partir das coordenadas
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            slope = (y2 - y1) / (x2 - x1)
            if slope > 0 and slope < math.inf:
                right_lines.append(line)
            elif slope < 0 and slope > -math.inf:
                left_lines.append(line)
                
    left_avg_slope, left_avg_intercept = find_median_slope_intercept(left_lines)
    right_avg_slope, right_avg_intercept = find_median_slope_intercept(right_lines)
    left_min_y = get_min_y(left_lines)
    left_max_y = img.shape[0]
    right_min_y = get_min_y(right_lines)
    right_max_y = img.shape[0]
    left_min_x = (left_min_y - left_avg_intercept) / left_avg_slope
    left_max_x = (left_max_y - left_avg_intercept) / left_avg_slope
    right_min_x = (right_min_y - right_avg_intercept) / right_avg_slope
    right_max_x = (right_max_y - right_avg_intercept) / right_avg_slope
    cv2.line(img, (int(left_min_x), int(left_min_y)), (int(left_max_x), int(left_max_y)), color, 5)
    cv2.line(img, (int(right_min_x), int(right_min_y)), (int(right_max_x), int(right_max_y)), color, 5)
                
def get_min_y(lines):
    min_y = math.inf
    for line in lines:
        for x1,y1,x2,y2 in line:
            min_y = min([min_y, y1, y2])
    return min_y
        
def find_median_slope_intercept(lines):
    slopes = []
    intercepts = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            # Diferença entre as coordenadas da linha
            slope = (y2 - y1) / (x2 - x1)
            slopes.append(slope)
            intercepts.append(y2 - slope*x2)
    median_slope = statistics.median(slopes)
    median_intercept = statistics.median(intercepts)
    return median_slope, median_intercept
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """ 
    'HoughLinesP' é uma técnica popular para detectar qualquer forma,
    se você puder representar essa forma na forma matemática.
    Ele pode detectar a forma mesmo que esteja quebrada ou distorcida um pouco
    https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(colored_section, original_image, alfa=0.8, beta=1., gama=0.):
    # original_image * alfa + img * beta + gama
    """
    'addWeighted' ajuda a sobrepor uma imagem de primeiro plano com uma aparência transparente enquanto tem uma imagem de fundo sobre esta.
    A transparência é geralmente o quarto canal de qualquer imagem. Essa máscara de transparência é conhecida como máscara alfa. 
    A imagem de primeiro plano está no canto superior esquerdo, enquanto a escala de cinza é a máscara alfa que está no canto superior direito.
    Para mesclar esta imagem, podemos adicionar pesos e definir a transparência e a translucidez das imagens.
    """
    return cv2.addWeighted(original_image, alfa, colored_section, beta, gama)

In [89]:
def extract_section_of_interest(image):
    gray = grayscale(image)
    blur_gray = gaussian_blur(gray, 3)
    canny_gray = canny(blur_gray, 128, 200)
    vertices = np.array([[(400, 350), (650, 350), (image.shape[1], image.shape[0]), (100, image.shape[0])]])
    """
    [400 350]
    [650 350]
    [960 540]
    [100 540]
    """
    roi_gray = region_of_interest(canny_gray, vertices)
    return roi_gray

In [90]:
def color_section_of_interest(masked_image):
    rho = 1
    theta = np.pi/180.0
    threshold = 10
    min_line_len = 5
    max_line_gap = 20
    line_gray = hough_lines(masked_image, rho, theta, threshold, min_line_len, max_line_gap)
    return line_gray

In [91]:
# Salva imagens extraindo região de interesse e colorindo
def save_image_section_of_interest():
    for image in images:
        image_name = image['name']
        masked_image = extract_section_of_interest(image['image'])
        masked_colored_image = color_section_of_interest(masked_image)
        cv2.imwrite('output/interest_area/' + image_name, masked_image)
        cv2.imwrite('output/colored_interest_area/' + image_name, masked_colored_image)

save_image_section_of_interest()

In [92]:
#Interpola as imagens
def interpolate_images():
    image_names = os.listdir('output/colored_interest_area')
    for name in image_names:
        original_image = mpimg.imread('input/' + name) 
        colored_image = mpimg.imread('output/colored_interest_area/' + name)
        interpolated_image = weighted_img(colored_image, original_image, alfa=0.8, beta=1., gama=0.)
        cv2.imwrite('output/interpolated_image/' + name, interpolated_image)
        
interpolate_images()